## Using Keras neural net to predict clicks on Russian Facebook and Instagram ads.
Unfortunately, this did not predict very well and still needs some work on the layering and optimization portion.

In [6]:
#This are some keras imports;
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer # Import the tokenizer to split the texts;
from keras.models import Sequential # Keras sequential approach to neural nets;
from keras.optimizers import Adam # Keras optimizer
from keras.layers import Dense # Some neural network layers;
from keras import backend as K # Allows us to calculate R-squared
from keras import layers
from sklearn.feature_extraction.text import HashingVectorizer

In [7]:
# Keras Neuralnet (did not predict)-----------------------------------------------
neuraldf = pd.read_csv('russiadata-bestmodel.csv')
neuraldf = pd.get_dummies(neuraldf, columns = ['Ad Landing Page '])
# neuraldf.to_csv('Neural Cleaned No nulls.csv')

text_data = neuraldf[["Ad Text "]].values.reshape(1,-1)[0] # have to use ndarray to mine text
tokenizer = Tokenizer(num_words=1000) # How to split the text... basically dummy codes each word
# 1000 words is a good amount... the more words I feed, the less accurate it gets
# (Tokenizer might also accept a list)

#figuring out what our 1000 words are
tokenizer.fit_on_texts(text_data) #Fitting data, gives each word a unique id

#creating a numpy arrary with the tokenized matrix for each post
#(aka actually applying our tokenizer to each piece of text)
wordmatrix = tokenizer.texts_to_matrix(text_data)
otherpredictors = neuraldf.drop('Ad Text ', axis=1) # Dropping this

#converting predictors to numpy array, the required format
#sucks, but we lose the variable names here
otherpredictorsmatrix = otherpredictors.values

#just adding all of our predictors together
# Concatenating the matrix of text and other predictors
X = np.concatenate((wordmatrix, otherpredictorsmatrix), axis=1) 
#this is the variable we're goint to try and predict
y = neuraldf[["Ad Clicks "]] #Getting the label as y;

# Initialize the model
model = Sequential()

#first, the Keras model needs to know the number of columns in the model
dimensions = X.shape[1]


#adding our first layer
model.add(Dense(50,input_dim=dimensions,activation="relu")) # * Input layer *

# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))

model.add(Dense(1, activation="softmax")) # * Output Layer *
    
model.summary()

#two key parameters here learning rate == lr the higher lr is, the more shortcuts it takes
#and the name of the optimizer itself
#https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2
optimizer = Adam(lr=.0001) #Preparing optimizer;

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


#Actually getting the model all ready to run
model.compile(loss="mean_squared_error", optimizer= optimizer, metrics=[coeff_determination]) 

#running the model
model.fit(X, y, batch_size=X.shape[0], epochs=5, shuffle=True, validation_split=0.3)#Training data;


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                107600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 112,751
Trainable params: 112,751
Non-trainable params: 0
_________________________________________________________________
Trai